# Encoder Decoder 

### Imports

In [2]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.utils import shuffle

### Importing Data

In [3]:
#df = pd.read_csv('ASL_English.csv')
df = pd.read_csv('..\Data\Double_Dataset_EnglishToGloss.csv')
df.head()

,English,ASL Gloss
0,I love to cook hamburgers on the grill.,G-R-I-L-L HAMBURGER COOK I LOVE.
1,I like to ice skate on our pond.,OUR P-O-N-D ICE SKATE I LIKE.
2,I like to be active and not sit and watch TV a...,ALL DAY I LIKE ACTIVE NOT SIT WATCH #TV.
3,Are you prepared for hurricane season?,HURRICANE S-E-A-S-O-N YOU READY PREPARE?
4,The dinner party was awkward because most of u...,DINNER PARTY AWKWARD WHY? MOST US NOT KNOW EA...


In [4]:
df.shape

(3116, 2)

### Preprocessing

Note:
- Replace the numbers/digits
- Check regarding Finger spellings
- Check if it is required to add start and end tokens to target sequences

In [5]:
replacements = {'1': " one ", '2':" two ", '3':" three ", '4':" four ", '5':" five ", '6':" six ", '7':" seven ", '8':" eight ", '9':" nine ", '0':" zero "}
df['English'] = df['English'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub('(\d)', lambda m: replacements[m.group()], x))

# Remove extra spaces
df['English'] = df['English'].apply(lambda x: x.strip())
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.strip())

# Lowercase all characters
df['English'] = df['English'].apply(lambda x: x.lower())
df['ASL Gloss'] = df['ASL Gloss'].apply (lambda x: x.lower())

# Removing double spaces
df['English'] = df['English'].apply(lambda x: x.replace('  ', ' '))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: x.replace('  ', ' '))

# Remove quotes # Might not need this
# df['English'] = df['English'].apply(lambda x: re.sub (r"'", '', x))
# df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: re.sub (r"'", '', x))

# Remove all special character
df['English'] = df['English'].apply(lambda x: ''.join (ch for ch in x if ch not in set(string.punctuation)))
df['ASL Gloss'] = df['ASL Gloss'].apply(lambda x: ''.join (ch for ch in x if ch not in set(string.punctuation)))

# Check if dataset has numbers
#print(df['English'].str.contains(r'\d').any())
#print(df['ASL Gloss'].str.contains(r'\d').any())

# Add tokens to target sequence
df['English'] = df['English'].apply(lambda x : 'START_ ' + x + ' _END')

In [6]:
df.head(10)

,English,ASL Gloss
0,START_ i love to cook hamburgers on the grill ...,grill hamburger cook i love
1,START_ i like to ice skate on our pond _END,our pond ice skate i like
2,START_ i like to be active and not sit and wat...,all day i like active not sit watch tv
3,START_ are you prepared for hurricane season _END,hurricane season you ready prepare
4,START_ the dinner party was awkward because mo...,dinner party awkward why most us not know each...
5,START_ my favorite author is me _END,my favorite author who me
6,START_ i love anything that has chocolate in i...,i love anything have chocolate inside
7,START_ in one nine four seven rhulin thomas be...,year one nine four seven rhulin thomas he firs...
8,START_ sean berdy is a deaf actor well known f...,sean berdy himself deaf actor famous why he in...
9,START_ did you see the movie about the life of...,you finish see movie about deaf woman herself ...


In [7]:
df.tail(10)

,English,ASL Gloss
3106,START_ how many pages is your resume _END,your resume howmany page
3107,START_ can you run your own business _END,can you manage yourself business
3108,START_ have you registered for school next yea...,nextyear school register finish you
3109,START_ what is your worst class _END,your worst class what
3110,START_ would you mind lending me five _END,five dollar dontmind lend me
3111,START_ does it have to be dark for you to slee...,for you sleep must dark
3112,START_ do you like gardening _END,you like takecareof plant you
3113,START_ how have you been doing _END,how you uptonow
3114,START_ what do you not care about _END,topic you dont care what
3115,START_ when did you start learning how to sign...,you start learn learn sign when


In [8]:
# df['ASL Gloss'].str.len().sort_values(ascending=False).head()
# df['English'].str.len().sort_values(ascending=False).head()

In [9]:
# Vocabulary of English
all_eng_words = set()
for eng in df['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of ASL 
all_ASL_words = set()
for asl in df['ASL Gloss']:
    for word in asl.split():
        if word not in all_ASL_words:
            all_ASL_words.add(word)

In [10]:
# Max Length of source sequence
lenght_list=[]
for l in df ['English']:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
print("Max length target: ", max_length_tar)

Max length target:  72


In [11]:
# Max Length of target sequence
lenght_list=[]
for l in df ['ASL Gloss']:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
print("Max length sorce: ", max_length_src)

Max length sorce:  41


In [12]:
input_words = sorted(list(all_ASL_words))
target_words = sorted(list(all_eng_words))

# Calculate Vocab size for both source and target
num_encoder_tokens = len(all_ASL_words) + 1
num_decoder_tokens = len(all_eng_words) + 1

num_encoder_tokens, num_decoder_tokens

(2036, 2085)

In [13]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

2086

In [14]:
# Create word to token dictionary for both source and target
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

# Create token to word dictionary for both source and target
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [15]:
shuffle(df).head(10)

,English,ASL Gloss
2009,START_ my mentor has helped me a lot _END,my mentor helpme a lot
210,START_ linda bove is a deaf actress she played...,linda bove herself deaf actress she act tv pro...
533,START_ where are you from _END,you from where
198,START_ my mothers baked potato soup is the bes...,my mother her baked potato soup champ best
121,START_ the soccer team lost but they still had...,soccer team allofthem lost doesnt matter still...
2599,START_ minnesota or missouri _END,minnesota missouri which
2947,START_ minnesota is in the north _END,minnesota where north
1165,START_ fortyeight people died from lightning i...,year two zero zero six four eight people died...
2218,START_ my family always gets together for inde...,every year independence day my family always g...
1343,START_ my grandmother was married in missouri ...,my grandmother married where missouri


Make a 90–10 train and test split and write a Python generator function to load the data in batches as follows:

In [17]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import tensorflow as tf

In [18]:
# Train - Test Split
X, y = df['ASL Gloss'], df['English']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((2804,), (312,))

Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [20]:
#X_train.to_pickle('Weights_ASL/X_train.pkl')
#X_test.to_pickle('Weights_ASL/X_test.pkl')

In [19]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X.iloc[j:j+batch_size], y.iloc[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

Encoder - Decoder Model Architecture

In [20]:
latent_dim = 50

In [21]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [22]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

# Use a softmax to generate a probability distribution over the target vocabulary for each time step
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 50)     101800      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 50)     104300      ['input_2[0][0]']                
                                                                                              

In [26]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

We train the network for 50 epochs with a batch size of 128.

In [27]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100
# epochs = 10

In [28]:
model.fit(generate_batch(X_test, y_test, batch_size = batch_size),
            batch_size = batch_size,
            steps_per_epoch = train_samples//batch_size,
            epochs=epochs,
            validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
            validation_steps = val_samples//batch_size)

Epoch 1/100
21/21 [==============================] - 31s 924ms/step - loss: 7.6180 - acc: 0.1060 - val_loss: 7.5840 - val_acc: 0.1162
Epoch 2/100
21/21 [==============================] - 15s 707ms/step - loss: 7.3852 - acc: 0.1189 - val_loss: 6.9196 - val_acc: 0.1162
Epoch 3/100
21/21 [==============================] - 15s 694ms/step - loss: 6.3590 - acc: 0.1189 - val_loss: 5.9158 - val_acc: 0.1162
Epoch 4/100
21/21 [==============================] - 15s 721ms/step - loss: 5.6936 - acc: 0.1189 - val_loss: 5.6367 - val_acc: 0.1162
Epoch 5/100
21/21 [==============================] - 15s 705ms/step - loss: 5.5175 - acc: 0.1189 - val_loss: 5.5656 - val_acc: 0.1162
Epoch 6/100
21/21 [==============================] - 15s 702ms/step - loss: 5.4545 - acc: 0.1189 - val_loss: 5.5289 - val_acc: 0.1162
Epoch 7/100
21/21 [==============================] - 16s 753ms/step - loss: 5.4160 - acc: 0.1189 - val_loss: 5.5014 - val_acc: 0.1162
Epoch 8/100
21/21 [==============================] - 15s 736ms

Always remember to save the weights

In [29]:
model.save_weights('Weights_ASL/nmt_weights_v4.h5')

Load the weights, if you close the application

In [30]:
model.load_weights('Weights_ASL/nmt_weights_v4.h5')

Inference Setup

In [31]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

Finally, we generate the output sequence by invoking the above setup in a loop as follows

Decode sample sequeces

In [32]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Evaluation on Train Dataset

In [33]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [34]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 32ms/step
Input ASL sentence: snow
Actual English Translation:  snowing 
Predicted English Translation:  its in the white 


In [35]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 32ms/step
Input ASL sentence: last christmas my cruise fancy gingerbread house i saw
Actual English Translation:  i saw a fancy gingerbread house on my cruise last christmas 
Predicted English Translation:  my was a a a was a nine 


In [36]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 31ms/step
Input ASL sentence: i love typing
Actual English Translation:  i love typing 
Predicted English Translation:  i love to 


In [37]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input ASL sentence:', X_train.iloc[k:k+1].values[0])
print('Actual English Translation:', y_train.iloc[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 31ms/step
Input ASL sentence: today weather forecast tornado
Actual English Translation:  todays weather forecast is a tornado 
Predicted English Translation:  the weather is is in the day 


In [38]:
asl_sentence = []
true_eng_trans = []
pred_eng_trans = []

In [39]:
for i in range(10):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    asl_sentence.append(X_train.iloc[k:k+1].values[0])
    true_eng_trans.append(y_train.iloc[k:k+1].values[0][6:-4])
    pred_eng_trans.append(decoded_sentence[:-4])

1/1 [==============================] - 0s 32ms/step


In [40]:
for i in range(10):
    print('Input ASL sentence:', asl_sentence[i])
    print('Actual English Translation:', true_eng_trans[i])
    print('Predicted English Translation:', pred_eng_trans[i])
    print()

Input ASL sentence: me go library get what good books
Actual English Translation:  i got good books from the library 
Predicted English Translation:  i like to my stove or 

Input ASL sentence: school nurse always busy
Actual English Translation:  the school nurse is always busy 
Predicted English Translation:  my grandfather is in in school 

Input ASL sentence: thanksgiving my cousin favorite holiday
Actual English Translation:  my cousins favorite holiday is thanksgiving 
Predicted English Translation:  my grandfather is in my from 

Input ASL sentence: i married nov two three year two zero two one 
Actual English Translation:  i got married november two three  two zero two one  
Predicted English Translation:  my was a nine one nine one nine one nine 

Input ASL sentence: today hawaii sun eight two degrees
Actual English Translation:  today in hawaii will be sunny and eight two degrees 
Predicted English Translation:  the is at the zero nine one years 

Input ASL sentence: my docto